## Insight for Cab investment firm

### Business problem:

XYZ is a private firm in US. Due to remarkable growth in the Cab Industry in last few years and multiple key players in the market, it is planning for an investment in Cab industry and as per their Go-to-Market(G2M) strategy they want to understand the market before taking final decision.

## Data Collection
### Load data sets and view structure

In [ ]:
import pandas as pd

# Load the datasets
cab_data = pd.read_csv('Cab_Data.csv')
customer_id_data = pd.read_csv('Customer_ID.csv')
transaction_id_data = pd.read_csv('Transaction_ID.csv')
city_data = pd.read_csv('City.csv')

# Display the first few rows of each dataset to understand their structure
cab_data.head(), customer_id_data.head(), transaction_id_data.head(), city_data.head()

In [ ]:
# Convert 'Date of Travel' from integer to datetime format
cab_data['Date of Travel'] = pd.to_datetime(cab_data['Date of Travel'], origin='1899-12-30', unit='D')

# Filter data based on the given time period
start_date = '2016-01-31'
end_date = '2018-12-31'
filtered_cab_data = cab_data[(cab_data['Date of Travel'] >= start_date) & (cab_data['Date of Travel'] <= end_date)]

# Display the first few rows of the filtered data
filtered_cab_data.head(15)

In [ ]:
# Check for missing values in all datasets
missing_values = {
    'Cab_Data': filtered_cab_data.isnull().sum(),
    'Customer_ID': customer_id_data.isnull().sum(),
    'Transaction_ID': transaction_id_data.isnull().sum(),
    'City': city_data.isnull().sum()
}

# Check for duplicates in all datasets
duplicates = {
    'Cab_Data': filtered_cab_data.duplicated().sum(),
    'Customer_ID': customer_id_data.duplicated().sum(),
    'Transaction_ID': transaction_id_data.duplicated().sum(),
    'City': city_data.duplicated().sum()
}
missing_values, duplicates

In [ ]:
# Joining the datasets based on common identifiers

# Joining Cab_Data with Transaction_ID on 'Transaction ID'
merged_data = pd.merge(filtered_cab_data, transaction_id_data, on='Transaction ID', how='left')

# Joining the merged data with Customer_ID on 'Customer ID'
merged_data = pd.merge(merged_data, customer_id_data, on='Customer ID', how='left')

# Display the first few rows of the merged dataset
merged_data.head()

## Cab Service Metrics: Distance, Pricing, Costs, and Company Popularity

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Set the style of the visualization
sns.set_theme(style="darkgrid", palette="pastel")

# Create a figure and a set of subplots
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))

# Plot distribution of 'KM Travelled'
sns.kdeplot(merged_data['KM Travelled'], fill=True, ax=axs[0, 0], shade_lowest=False)
axs[0, 0].set_title('Distribution of KM Travelled')

# Plot distribution of 'Price Charged'
sns.kdeplot(merged_data['Price Charged'], fill=True, ax=axs[0, 1], shade_lowest=False)
axs[0, 1].set_title('Distribution of Price Charged')

# Plot distribution of 'Cost of Trip'
sns.kdeplot(merged_data['Cost of Trip'], fill=True, ax=axs[1, 0], shade_lowest=False)
axs[1, 0].set_title('Distribution of Cost of Trip')

# Plot count of customers for each cab company
sns.countplot(x='Company', data=merged_data, ax=axs[1, 1], palette=['pink', 'yellow'])
axs[1, 1].set_title('Count of Customers for Each Cab Company')

# Adjust the layout of the plots
plt.tight_layout()
plt.show()

### Analysis of Cab Service Metrics: 

The first three histograms provide insights into the distribution of key metrics for cab rides: distance traveled, price charged, and the cost of the trip. Most rides fall within the 10-30 KM range, indicating short to medium-length trips are predominant. The distribution of 'Price Charged' reveals that most fares are clustered between 0 to 500 USD, with a few instances of higher charges, possibly for longer rides or premium services. The 'Cost of Trip' histogram shows a similar pattern to 'KM Travelled', suggesting a direct correlation between distance and cost.

The bar chart on the bottom right showcases the popularity of the two cab companies. Yellow Cab has a significantly higher customer count compared to Pink Cab. This could be attributed to various factors such as wider availability, better service, or more aggressive marketing. However, the sheer difference in customer counts suggests that Yellow Cab holds a dominant position in the market, making it a potential frontrunner for investment opportunities.

## Market Share Distribution: Yellow Cab vs. Pink Cab

In [ ]:
# Analyzing which company has the maximum cab users during the specified time period
company_users = merged_data['Company'].value_counts()

# Plotting
colors = ['pink', 'yellow']
explode = (0.1, 0)  # explode 1st slice
labels = ['Pink Cab', 'Yellow Cab']
plt.figure(figsize=(10, 6))
plt.pie(company_users, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=180)
plt.title('Distribution of Users between Pink Cab and Yellow Cab')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

### Analysis of Cab Users by Company

The pie chart provides a clear representation of the market share between the two cab companies over the specified time period. Yellow Cab dominates the market with a substantial 77.2% of the total users, more than three times the user base of Pink Cab, which holds only 22.8%. This significant difference indicates that Yellow Cab has a more extensive reach and possibly offers services that are more aligned with the preferences or needs of the majority of cab users.

However, a larger user base doesn't necessarily equate to higher profitability. While Yellow Cab has captured a more significant portion of the market, it's essential to delve deeper into operational costs, pricing strategies, and customer satisfaction to determine the overall profitability and sustainability of each company. The data suggests that any potential investor or stakeholder should consider both the market share and the underlying factors contributing to these numbers before making informed decisions.

## Calculate profits for each cab company

In [ ]:
# Calculate profit for each trip
merged_data['Profit'] = merged_data['Price Charged'] - merged_data['Cost of Trip']

# Calculate average profit for each cab company
average_profit = merged_data.groupby('Company')['Profit'].mean()
average_profit

## Payment mode distribution for both companies

In [ ]:
import matplotlib.pyplot as plt

# Data preparation
payment_data = merged_data.groupby(['Company', 'Payment_Mode']).size().reset_index(name='Counts')
yellow_data = payment_data[payment_data['Company'] == 'Yellow Cab']
pink_data = payment_data[payment_data['Company'] == 'Pink Cab']

# Plotting
fig, ax = plt.subplots(1, 2, figsize=(15, 7))

# Yellow Cab
ax[0].pie(yellow_data['Counts'], labels=yellow_data['Payment_Mode'], autopct='%1.1f%%', startangle=90, colors=['#FFD700', '#FF6347'], wedgeprops=dict(width=0.3))
ax[0].set_title('Yellow Cab Payment Mode Distribution', fontsize=15)

# Pink Cab
ax[1].pie(pink_data['Counts'], labels=pink_data['Payment_Mode'], autopct='%1.1f%%', startangle=90, colors=['#FFC0CB', '#D8BFD8'], wedgeprops=dict(width=0.3))
ax[1].set_title('Pink Cab Payment Mode Distribution', fontsize=15)

plt.tight_layout()
plt.show()

### Analysis of Cab Service Payment Trends

The donut charts vividly illustrate the payment preferences of customers across two major cab service providers: Pink Cab and Yellow Cab. For both companies, card payments dominate, underscoring a broader trend in consumer behavior favoring digital transactions. Specifically, Yellow Cab sees an overwhelming preference for card payments, with a staggering 85% of its customers opting for this mode. Pink Cab, while also leaning towards card transactions, has a slightly more balanced distribution, with approximately 70% of its users choosing cards.

This divergence in payment methods between the two companies might hint at differing customer demographics or operational strategies. For instance, Yellow Cab's higher card payment percentage could suggest a more tech-savvy user base or perhaps more effective promotions encouraging digital payments. On the other hand, Pink Cab's relatively higher cash transactions might indicate a customer base that values traditional payment methods or operates in areas with less digital penetration. For both companies, understanding these nuances is essential to tailor their payment infrastructure and marketing strategies effectively.

## Analysis of Quarterly Revenue and Profit for Yellow Cab and Pink Cab

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Grouping by quarter and calculating revenue and profit
quarterly_data = merged_data.groupby([merged_data['Date of Travel'].dt.to_period('Q'), 'Company'])[['Price Charged', 'Cost of Trip']].sum().reset_index()
quarterly_data['Profit'] = quarterly_data['Price Charged'] - quarterly_data['Cost of Trip']

# Plotting
plt.figure(figsize=(14, 8))
for company, color in [('Yellow Cab', 'yellow'), ('Pink Cab', 'pink')]:
    company_data = quarterly_data[quarterly_data['Company'] == company]
    plt.plot(company_data['Date of Travel'].astype(str), company_data['Price Charged']/1e6, label=f'Revenue {company}', color=color, linewidth=2.5)
    plt.plot(company_data['Date of Travel'].astype(str), company_data['Profit']/1e6, label=f'Profit {company}', linestyle='--', color=color, linewidth=2.5)

# Formatting the plot
plt.title('Quarterly Revenue and Profit by Company', fontsize=16, fontweight='bold')
plt.xlabel('Quarter', fontsize=15)
plt.xticks(rotation=45, fontsize=13)
plt.ylabel('Amount (in millions USD)', fontsize=15)
plt.yticks(fontsize=13)
plt.legend(loc='upper left', fontsize=15)
plt.grid(True, which='both', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

### Analysis
Yellow Cab consistently dominates the market, outperforming Pink Cab in both revenue and profit across all quarters, indicating a stronger market presence and possibly a broader customer base or superior pricing strategies. Both companies exhibit seasonal trends in their earnings, with certain quarters like Q2 and Q4 showing higher revenues, potentially due to factors like holidays or promotional campaigns.

From an investment perspective, Yellow Cab appears as the more stable choice, showcasing consistent profitability. However, Pink Cab's fluctuating profit margins suggest potential growth opportunities if operational challenges are addressed. While Yellow Cab seems to be the safer bet, integrating external data like US holiday patterns could offer deeper insights, making Pink Cab an intriguing option for diversified investment.

## Gender Distribution Across Cab Companies

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Data preparation
gender_data = merged_data.groupby(['Company', 'Gender']).size().reset_index(name='Counts')

# Create a pie chart for each company
companies = ['Yellow Cab', 'Pink Cab']
colors = ['#FFD700', '#FF69B4']  # Yellow for male, Pink for female
fig, axs = plt.subplots(1, 2, figsize=(15, 7))

for ax, company in zip(axs, companies):
    company_data = gender_data[gender_data['Company'] == company]
    ax.pie(company_data['Counts'], labels=company_data['Gender'], colors=colors, autopct='%1.1f%%', startangle=140, wedgeprops=dict(width=0.3))
    ax.set_title(company)

plt.suptitle('Gender Distribution Across Cab Companies', fontsize=16)
plt.show()

### Analysis of Gender distribution Pink Cab Vs. Yellow Cab Company
The pie charts provide a clear visualization of the gender distribution across both Yellow Cab and Pink Cab companies. For the Yellow Cab company, the distribution is almost even, with males representing approximately 50.7% and females constituting 49.3%. This suggests that the Yellow Cab service is equally preferred by both genders. On the other hand, the Pink Cab company shows a slightly higher preference among males, with males making up 52.3% and females 47.7% of the customer base. The slight male dominance in Pink Cab's customer base might be attributed to various factors, including marketing strategies or service offerings. However, it's essential to note that both companies have a relatively balanced gender distribution, indicating that cab services are universally utilized across genders. The pie chart visualization offers a direct comparison and an immediate understanding of the gender split for each company.






## Distribution of Customers by Age Group and Cab Company

In [ ]:
import numpy as np
# Define age groups
bins = [18, 30, 40, 50, 60, 70, 80]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70+']
merged_data['Age Group'] = pd.cut(merged_data['Age'], bins=bins, labels=labels, right=False)

# Plotting
plt.figure(figsize=(12, 7))
sns.countplot(data=merged_data, x='Age Group', hue='Company', palette=['pink', 'yellow'])
plt.title('Distribution of Customers by Age Group and Cab Company', fontsize=16)
plt.xlabel('Age Group', fontsize=14)
plt.ylabel('Number of Customers', fontsize=14)
plt.legend(title='Cab Company')
plt.show()

### Analysis of Customer Distribution by Age Group and Cab Company

The visualization showcases the distribution of customers across various age groups for both Yellow Cab and Pink Cab companies. A few key observations can be made:

Younger Demographic Preference: The age group '18-29' has the highest number of customers for both cab companies, indicating that younger individuals are more inclined to use cab services. This could be attributed to factors like urban lifestyle, tech-savviness, or the convenience of app-based cab bookings. While both companies have a significant presence in this age bracket, Yellow Cab dominates with a considerably higher customer count.

Decline in Older Age Groups: As the age group increases, there's a noticeable decline in the number of customers for both companies. The '70+' age group has the least number of customers, suggesting that older individuals might prefer other modes of transportation or are less frequent travelers. However, in every age group, Yellow Cab maintains a lead over Pink Cab, emphasizing its broader appeal or market presence across all age demographics.

## Merge dataset to analyze based on cities

In [ ]:
# Joining the merged data with City on 'City' column
merged_data_with_city = pd.merge(merged_data, city_data, on='City', how='left')

# Display the first few rows of the final dataset
merged_data_with_city.head(15)

## Profit distribution by city and cab company

### Analysis of Cab Service Payment Trends

The pie chart vividly illustrates the payment preferences of customers across two major cab service providers: Pink Cab and Yellow Cab. For both companies, card payments dominate, underscoring a broader trend in consumer behavior favoring digital transactions. Specifically, Yellow Cab sees an overwhelming preference for card payments, with a staggering 85% of its customers opting for this mode. Pink Cab, while also leaning towards card transactions, has a slightly more balanced distribution, with approximately 70% of its users choosing cards.

This divergence in payment methods between the two companies might hint at differing customer demographics or operational strategies. For instance, Yellow Cab's higher card payment percentage could suggest a more tech-savvy user base or perhaps more effective promotions encouraging digital payments. On the other hand, Pink Cab's relatively higher cash transactions might indicate a customer base that values traditional payment methods or operates in areas with less digital penetration. For both companies, understanding these nuances is essential to tailor their payment infrastructure and marketing strategies effectively.

In [ ]:
# Calculate average profit for each city and cab company
merged_data_with_city['Profit'] = merged_data_with_city['Price Charged'] - merged_data_with_city['Cost of Trip']
citywise_profit = merged_data_with_city.groupby(['City', 'Company'])['Profit'].mean().reset_index()

# Visualize the average profit for each city and cab company
plt.figure(figsize=(15, 8))
sns.barplot(x='City', y='Profit', hue='Company', data=citywise_profit)
plt.title('Average Profit by City and Cab Company')
plt.xticks(rotation=45)
plt.show()

### Analysis profit by city and cab company
The visualization showcases the average profit margins for both Yellow and Pink cab companies across various cities. It's evident that Yellow Cab consistently outperforms Pink Cab in terms of profit across almost all cities. Cities like New York, Los Angeles, and San Francisco, which are major urban centers, show the highest profit margins for Yellow Cab. This could be attributed to a higher demand, longer trips, or a more established presence in these cities. On the other hand, Pink Cab, while still profitable, lags behind in these major cities. It's worth noting that in some cities like Pittsburgh and Sacramento, Pink Cab's profit is closer to Yellow Cab, suggesting that there might be opportunities or market dynamics in these cities that Pink Cab can further capitalize on.

## Customer distribution by city and cab company

In [ ]:
import seaborn as sns

# Set the style of the visualization
sns.set(style="whitegrid")

# Create a treemap
fig, ax = plt.subplots(figsize=(15, 10))
city_company_data = merged_data_with_city.groupby(['City', 'Company']).size().reset_index(name='Counts')
city_company_data = city_company_data.pivot('City', 'Company', 'Counts')
sns.heatmap(city_company_data, cmap='YlGnBu', annot=True, fmt='g', linewidths=.5, ax=ax)

ax.set_title('Customer Distribution by City and Cab Company', fontsize=16)
plt.show()

### Analysis of Customer Distribution by City and Cab Company

The heatmap provides a clear visualization of the customer distribution across various cities for both Pink Cab and Yellow Cab companies. At a glance, it's evident that Yellow Cab has a more extensive customer base in almost all cities compared to Pink Cab. Cities like New York, Chicago, and Los Angeles have the highest concentration of Yellow Cab users, with New York leading by a significant margin. This dominance in major cities indicates Yellow Cab's strong market presence and possibly more extensive operations in these regions.

On the other hand, Pink Cab has a relatively smaller customer base in these cities, but it's worth noting that they still maintain a presence. The difference in customer counts between the two companies is especially pronounced in cities like San Diego and Seattle. For investors or stakeholders, this heatmap can serve as a guide to understand market penetration and potential growth areas. While Yellow Cab seems to have captured the major cities, Pink Cab might find opportunities in expanding its services or focusing on niche markets within these cities.

## Comparative Profit Landscape: City-wise Breakdown for Yellow Cab and Pink Cab"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extracting data for the plot
citywise_profit = merged_data_with_city.groupby(['City', 'Company'])['Profit'].sum().unstack().fillna(0)
cities_sorted_by_profit = citywise_profit.sum(axis=1).sort_values(ascending=False).index
citywise_profit = citywise_profit.loc[cities_sorted_by_profit]

# Plotting
fig, ax = plt.subplots(figsize=(15, 8))
citywise_profit.divide(1e6).plot(kind='area', stacked=True, ax=ax, alpha=0.7)

# Formatting the plot
ax.set_title('Comparative Profit Landscape: City-wise Breakdown for Yellow Cab and Pink Cab', fontsize=16, fontweight='bold')
ax.set_xlabel('City', fontsize=15)
ax.set_ylabel('Total Profit (in millions USD)', fontsize=15)
ax.tick_params(axis='x', rotation=45)
ax.legend(title='Cab Company', fontsize=12)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

### Analysis of total profit by city and cab company
The stacked area chart provides a comprehensive view of the profit landscape across various cities for both Yellow Cab and Pink Cab. It's evident that certain cities, such as New York, Chicago, and Los Angeles, are major profit centers for both companies, with Yellow Cab consistently outperforming Pink Cab. The chart also reveals that while Yellow Cab has a broader profit distribution across multiple cities, Pink Cab's profit is more concentrated in specific regions. This could be attributed to Yellow Cab's expansive operational presence or more effective market strategies. For potential investors, understanding these city-wise profit dynamics is crucial, as it highlights the regions where each company has a stronghold and where there might be opportunities for growth or increased competition.

## Conclusion
Based on the data analysis of the two cab companies over the specified time period, several key insights emerge

### Market Presence:
Yellow Cab has a dominant presence across most cities, especially in major urban centers. This suggests a well-established network and potentially a broader customer base.

### Profitability: 
While both companies have shown growth, Yellow Cab consistently outperforms Pink Cab in terms of profitability. This could be attributed to better operational efficiencies, pricing strategies, or a combination of factors.

### Customer Preferences:
The analysis of payment modes and customer demographics provides insights into the preferences and behaviors of the customers of each company. Digital payments dominate, indicating a shift towards cashless transactions.

### Seasonal Trends:
Both companies exhibit seasonal trends in their earnings. Understanding these patterns can help in strategizing promotional campaigns or offers to boost revenues during peak seasons.

In conclusion, Yellow Cab appears to be the more stable and profitable choice for investment. However, Pink Cab, with its unique customer demographics and potential growth opportunities, offers a chance for diversified investment. For XYZ, the decision should factor in both the current market dynamics and future growth potential of the cab industry.